Generating Hidden States

Hidden States - Sunny and Cloudy
Observable States - Happy and Sad

Transition Probabilities

Sunny -- 0.8 --> Sunny

Sunny -- 0.2 --> Cloudy

Cloudy -- 0.4 --> Sunny

Cloudy -- 0.6 --> Cloudy


Emission Probabilities

Sunny -- 0.7 --> Happy

Sunny -- 0.3 --> Sad

Cloudy -- 0.4 --> Happy

Cloudy -- 0.6 --> Sad

In [1]:
import numpy as np
import pandas as pd
from hmmlearn import hmm

In [2]:
# sample_size = 500
# hidden_floats = np.random.uniform(0, 1, sample_size)
# hidden_states = ['Sunny', 'Cloudy']
# hidden_seq = []

# if hidden_floats[0] < 0.5:
#     hidden_seq.append(hidden_states[0])
# else:
#     hidden_seq.append(hidden_states[1])
    
# for i in range(1, sample_size):
    
#     if hidden_seq[i - 1] == hidden_states[0]:
#         if hidden_floats[i] < 0.8:
#             hidden_seq.append(hidden_states[0])
#         else:
#             hidden_seq.append(hidden_states[1])
            
#     elif hidden_seq[i - 1] == hidden_states[1]:
#         if hidden_floats[i] < 0.4:
#             hidden_seq.append(hidden_states[0])
#         else:
#             hidden_seq.append(hidden_states[1])

In [3]:
# obs_floats = np.random.uniform(0, 1, sample_size)
# obs_states = ['Happy', 'Sad']
# obs_seq = []

# for i in range(sample_size):
#     if hidden_seq[i] == hidden_states[0]:
#         if obs_floats[i] < 0.7:
#             obs_seq.append(obs_states[0])
#         else:
#             obs_seq.append(obs_states[1])
    
#     elif hidden_seq[i] == hidden_states[1]:
#         if obs_floats[i] < 0.4:
#             obs_seq.append(obs_states[0])
#         else:
#             obs_seq.append(obs_states[1])

In [4]:
# pd.DataFrame({'Hidden States': hidden_seq,
#              'Observations': obs_seq})

In [5]:
# with open('hmm_sim.txt', 'w') as f:
#     for state in obs_seq:
#         f.write("%s\n" % state)

In [6]:
# with open('hmm_hidden_sim.txt', 'w') as f:
#     for state in hidden_seq:
#         f.write("%s\n" % state)

Learning HMM

In [7]:
# with open('left_right.txt', 'r') as f:
#     line = f.readline()
#     obs = []
#     while line != '':
#         obs += list(''.join(x for x in line if x.isalpha()))
#         line = f.readline()

In [8]:
with open('hmm_obs_sim.txt', 'r') as f:
    line = f.readline()[:-1]
    obs = []
    while line != '':
        obs.append(line)
        line = f.readline()[:-1]
X = obs

In [9]:
X = []
for x in obs:
    if x == 'Happy':
        X.append([1])
    else:
        X.append([0])

In [10]:
remodel = hmm.MultinomialHMM(n_components=2, n_iter=1000)
# remodel = hmm.GaussianHMM(n_components=2, n_iter=1000)

remodel.startprob_ = np.array([0.5,0.5])
remodel.transmatrix_ = np.array([[0.8,0.2],
                                 [0.4,0.6]])
remodel.emissionprob_ = np.array([[0.7,0.3],
                                  [0.4,0.6]])

remodel.fit(X)
Z2 = remodel.predict(X)

Even though the 'startprob_' attribute is set, it will be overwritten during initialization because 'init_params' contains 's'


In [11]:
zero_after_one = 0
zero_after_zero = 0
one_after_one = 0
one_after_zero = 0
for i in range(1, len(Z2)):
    if Z2[i] == 0 and Z2[i - 1] == 1:
        zero_after_one += 1
    elif Z2[i] == 0 and Z2[i - 1] == 0:
        zero_after_zero += 1
    elif Z2[i] == 1 and Z2[i - 1] == 1:
        one_after_one += 1
    elif Z2[i] == 1 and Z2[i - 1] == 0:
        one_after_zero += 1

after_one = zero_after_one + one_after_one
after_zero = zero_after_zero + one_after_zero

trans_probs = [zero_after_one / after_one,
              one_after_one / after_one,
              zero_after_zero / after_zero,
              one_after_zero / after_zero]

trans_probs

[0.3948220064724919,
 0.6051779935275081,
 0.3631578947368421,
 0.6368421052631579]

In [12]:
happy_after_one = 0
happy_after_zero = 0
sad_after_one = 0
sad_after_zero = 0

for i in range(len(Z2)):
    if Z2[i] == 1 and obs[i] == 'Happy':
        happy_after_one += 1
    elif Z2[i] == 0 and obs[i] == 'Happy':
        happy_after_zero += 1
    elif Z2[i] == 1 and obs[i] == 'Sad':
        sad_after_one += 1
    elif Z2[i] == 0 and obs[i] == 'Sad':
        sad_after_zero += 1

emission_probs = [happy_after_one / after_one,
                  sad_after_one / after_one,
                  happy_after_zero / after_zero,
                  sad_after_zero / after_zero
                 ]

emission_probs     

[1.0, 0.0, 0.0, 1.0052631578947369]

In [13]:
pd.DataFrame({'hidden': Z2,
             'obs': obs})

,hidden,obs
0,1,Happy
1,1,Happy
2,1,Happy
3,0,Sad
4,0,Sad
...,...,...
495,1,Happy
496,1,Happy
497,1,Happy
498,1,Happy


In [14]:
#multiple hidden states
#continuous observed state

In [15]:
# Simple Collection of 30 observations

# In relation to study: hidden states are action (sitting, walking, running, biking, etc.)
# Observed state is level of activity

# Method of collection: All hidden states randomly collected first, 
# and then all observed states calculated from hidden states

# Hidden state probabilities: 50% chance to stay in current hidden state,
#                             30% chance to go to next hidden state (i.e. sit to walk, walk to run, run to bike, bike to sit)
#                             10% chance each for the remaining two hidden states

# Observed state calculated from normal distributions based on hidden state:
# Sit ~ N(5, 2)
# Walk ~ N(10, 2.5)
# Run ~ N(15, 3)
# Bike ~ N(20, 2)


new_hidden_states = ["Sit", "Run", "Bike", "Bike", "Sit", "Sit", "Sit", "Sit", "Walk", "Run",
                     "Bike", "Bike", "Bike", "Bike", "Sit", "Sit", "Sit", "Sit", "Bike", "Walk",
                     "Walk", "Sit", "Sit", "Walk", "Walk", "Run", "Run", "Run", "Run", "Run"]

new_hidden_states = pd.Series(new_hidden_states)

new_observed_states = [5.01, 16.82, 20.28, 21.02, 5.09, 7.43, 6.07, 4.20, 13.5, 13.34,
                       19.88, 21.30, 20.17, 18.94, 5.88, 2.39, 7.94, 3.50, 17.47, 9.03,
                       13.7, 4.43, 9.34, 7.90, 9.04, 13.93, 10.33, 14.20, 15.18, 15.07]

In [21]:
X = []
for x in new_observed_states:
    X.append([x])

remodel = hmm.GaussianHMM(n_components=4, covariance_type="diag", n_iter=1000)

remodel.startprob_ = np.array([1,0,0,0])

remodel.transmatrix_ = np.array([[0.5,0.3,0.1,0.1],
                                 [0.1,0.5,0.3,0.1],
                                 [0.1,0.1,0.5,0.3],
                                 [0.3,0.1,0.1,0.5]])

remodel.means_ = np.array([5, 10, 15, 20])

remodel.covar_ = np.array([2, 2.5, 3, 2])


remodel.fit(X)
Z2 = remodel.predict(X)

Even though the 'startprob_' attribute is set, it will be overwritten during initialization because 'init_params' contains 's'
Even though the 'means_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'm'


In [28]:
# These values change every time. I change the numbers to correspond to the correct state, 
# but I don't know how to handle this without hard-coding it in every time

def assign_hidden_states(result):
    if result == 0:
        return "Sit"
    if result == 2:
        return "Walk"
    if result == 3:
        return "Run"
    if result == 1:
        return "Bike"
        
hidden_state_predictions = pd.Series(Z2).apply(lambda x: assign_hidden_states(x))

In [29]:
hidden_state_predictions

0      Sit
1      Run
2     Bike
3     Bike
4      Sit
5      Sit
6      Sit
7      Sit
8      Run
9      Run
10    Bike
11    Bike
12    Bike
13    Bike
14     Sit
15     Sit
16     Sit
17     Sit
18     Run
19    Walk
20    Walk
21    Walk
22    Walk
23    Walk
24    Walk
25    Walk
26    Walk
27     Run
28     Run
29     Run
dtype: object

In [30]:
new_hidden_states

0      Sit
1      Run
2     Bike
3     Bike
4      Sit
5      Sit
6      Sit
7      Sit
8     Walk
9      Run
10    Bike
11    Bike
12    Bike
13    Bike
14     Sit
15     Sit
16     Sit
17     Sit
18    Bike
19    Walk
20    Walk
21     Sit
22     Sit
23    Walk
24    Walk
25     Run
26     Run
27     Run
28     Run
29     Run
dtype: object

In [32]:
correct = hidden_state_predictions == new_hidden_states

In [35]:
print("Accuracy: " + str(sum(correct)/len(new_hidden_states)*100) + "%")

Accuracy: 80.0%
